In [1]:
import os
%load_ext autoreload
%autoreload 2

""""
For extracting and saving DFallpa objects
"""

'"\nFor extracting and saving DFallpa objects\n'

# Load a DFallpa

In [ ]:
# Char, 
# animal = "Pancho"
# date = 230623
# do_combine = True

# Single prim, novels
animal = "Diego"
# date = 231211
# date = 240619
# animal = "Diego"
date = 231211
# date = 240605
# date = 230615
do_combine = False

if do_combine:
    # COMBINE trial and stroke
    dir_suffix = "test"
    question = None
    # q_params = None
    which_level = None
    q_params = {
        "effect_vars": ["seqc_0_shape", "seqc_0_loc"]
    }
    
    combine_trial_and_stroke = True
    
    # PIG
    # question_trial = "PIG_BASE_trial"
    # question_stroke = "PIG_BASE_stroke"
    # check_that_locs_match = True
    
    # CHAR
    question_trial = "PIG_BASE_trial"
    question_stroke = "PIG_BASE_stroke"
    check_that_locs_match = False
    check_that_shapes_match = False

    HACK_RENAME_SHAPES = "CHAR" in question_trial
    
    events_keep_trials = ["03_samp", "05_first_raise"]

else:
    # DONT COMBINE, use questions.
    # question = "SS_shape"
    question = "CHAR_BASE_stroke"
    # question = "CHAR_BASE_trial"
    # question = "SP_shape_loc"
    # question = "SP_BASE_stroke"
    # question = "SP_BASE_trial"
    # question = "PIG_BASE_trial"
    # question = "RULE_BASE_trial"
    # question = "PIG_BASE_stroke"
    # question = "RULE_ANBMCK_STROKE"
    
    # which_level = "trial" # Doesnt matter
    which_level = "stroke" # Doesnt matter
    # which_level = "substroke" # Doesnt matter

    dir_suffix = question
    combine_trial_and_stroke = False
    # Load q_params
    from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
    q_params = rsagood_questions_dict(animal, date, question)[question]

    HACK_RENAME_SHAPES = "CHAR" in question
    events_keep = ["00_stroke"]
    # events_keep = ["03_samp", "05_first_raise", "06_on_strokeidx_0"]


############### PARAMS
exclude_bad_areas = True
SPIKES_VERSION = "kilosort_if_exists" # since Snippets not yet extracted for ks
# SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
combine_into_larger_areas = True


list_time_windows = [(-0.8, 1.2)]
# list_time_windows = [(-0.8, 1.4)]
# list_time_windows = [(-0.8, 1.25)]
# list_time_windows = [(-1., 1.6)]
# events_keep = None
# events_keep = ["03_samp", "05_first_raise", "06_on_strokeidx_0"]
# events_keep = ["03_samp", "05_first_raise", "06_on_strokeidx_0"]
fr_normalization_method = None
# fr_normalization_method = "across_time_bins"

########################################## RUN

if combine_trial_and_stroke:
    from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper_combine_trial_strokes
    DFallpa = dfallpa_extraction_load_wrapper_combine_trial_strokes(animal, date, question_trial,
                                                                       question_stroke,
                                                list_time_windows, events_keep_trials=events_keep_trials,
                                               combine_into_larger_areas = combine_into_larger_areas,
                                               exclude_bad_areas=exclude_bad_areas,
                                                SPIKES_VERSION=SPIKES_VERSION,
                                                HACK_RENAME_SHAPES = HACK_RENAME_SHAPES,
                                               fr_normalization_method=fr_normalization_method,
                                                    check_that_shapes_match=check_that_shapes_match,
                                                check_that_locs_match=check_that_locs_match)
else:
    from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper
    DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                              which_level=which_level, events_keep=events_keep,
                                              combine_into_larger_areas = combine_into_larger_areas,
                                              exclude_bad_areas = exclude_bad_areas,
                                              SPIKES_VERSION = SPIKES_VERSION,
                                              HACK_RENAME_SHAPES = HACK_RENAME_SHAPES,
                                              fr_normalization_method=fr_normalization_method)

In [ ]:
DFallpa


In [ ]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
pa.Times

In [ ]:
dflab["shape_semantic_grp"]

In [ ]:
dflab["shape_semantic_grp"].value_counts()

In [ ]:
sorted(dflab.columns.tolist())

In [ ]:
for col in pa.Xlabels["trials"]:
    print(col, "   ====   ", type(pa.Xlabels["trials"][col].values[0]))

In [ ]:
sorted(pa.Xlabels["trials"]["seqc_0_shape"].unique().tolist())

# Eye tracking (Kedar)

In [ ]:
from neuralmonkey.classes.session import load_mult_session_helper
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper
from neuralmonkey.classes.snippets import load_and_concat_mult_snippets
from neuralmonkey.analyses.rsa import rsagood_questions_dict
import pythonlib
import pandas as pd
import os

In [ ]:
# get session object
animal = "Diego"
date_list = [230630]
RUN_CLUSTERFIX = False

question = "PIG_BASE_saccade_fix_on" # holds variety of prepropoessing steps to clean data, specificalyl for PIG data.
list_time_windows = [(-0.4, 0.4)] # to slice data including just within this time window (realtive to events)
##### note: fine for preparation period, but may want to change if diff period^
events_keep = ["00_fixon_preparation"]
combine_into_larger_areas = False
which_level = "saccade_fix_on"
SPIKES_VERSION = "kilosort_if_exists"

for date in date_list:
    MS = load_mult_session_helper(date, animal, spikes_version=SPIKES_VERSION)
    # sn = MS.SessionsList[session]

    if RUN_CLUSTERFIX==True:
        for sn in MS.SessionsList:
            session = sn.RecSession
            print("doing session: ", session)
            savedir = f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}-{session}"
            os.makedirs(savedir, exist_ok=True)
            sn.extract_and_save_clusterfix_results()
        
    DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                        which_level=which_level,
                                        events_keep = events_keep,
                                        combine_into_larger_areas=combine_into_larger_areas,
                                        HACK_RENAME_SHAPES=False,
                                        SPIKES_VERSION=SPIKES_VERSION)
    


In [ ]:
    os.makedirs(savedir_pkl, exist_ok=True)    
    savedir_pkl = f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}"
    DFallpa.to_pickle(savedir_pkl + "/dfallpa.pkl")

In [ ]:
DFallpa

In [ ]:
pa = DFallpa["pa"].values[2]

In [ ]:
pa.Chans

In [ ]:
sorted(pa.Xlabels["trials"].columns)

In [ ]:
pa.plotwrapper_smoothed_fr_split_by_label_and_subplots(pa.Chans[23], "shape-fixation", ("early-or-late-planning-period", "seqc_0_shape"))

# Saving DFallpa

### Save DFallpa as is

In [ ]:
# Save it
import pandas as pd
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa.pkl"
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa.pkl"
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa_2.pkl" # (tdt) (no norm)
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa_3.pkl" # (no norm)
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa_4.pkl" # (tdt)
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa_KS.pkl" # (kilosort)
# path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa_KS_nonorm.pkl" # (kilosort)

# Dan: tough decoding, syntax stuff.
# path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Dolnik/DFallpa-{animal}-{date}-{which_level}-tdt_nonorm.pkl"

# Xuan: Diego, char, good for testing tough shape decoding
# path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-{animal}-{date}-{which_level}-ks_nonorm.pkl"
path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-{animal}-{date}-{which_level}-{SPIKES_VERSION}-norm={fr_normalization_method}-combine={combine_into_larger_areas}.pkl"

pd.to_pickle(DFallpa, path)
print("Saved to:", path)

### Save raw data [Dolnik]

In [ ]:
animal = "Diego"
date = 240516
which_level = "trial"
SPIKES_VERSION = "kilosort_if_exists"
fr_normalization_method = None
combine_into_larger_areas = True

In [ ]:
SPIKES_VERSION = "kilosort_if_exists"
fr_normalization_method = None
combine_into_larger_areas = True
which_level = "trial"

In [ ]:
##### To save DFallPA
from neuralmonkey.classes.population_mult import data_extract_raw_and_save
from neuralmonkey.classes.population_mult import load_handsaved_wrapper
DFallpa, path = load_handsaved_wrapper("Diego", 230615, "trial", question="SP_BASE_trial", also_return_path=True)



In [ ]:
path

In [ ]:
savepath = '/lemur2/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-230615-trial-kilosort_if_exists-norm=None-combine=True-t1=-1.0-t2=1.8-quest=SP_BASE_trial-RAW.pkl'
data_extract_raw_and_save(DFallpa, savepath)
# pd.to_pickle(DFallpa, savepath)gm

In [ ]:
DFallpa


In [ ]:
# Load RAW (Dan saved)
import pandas as pd
path = "/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-240516-trial-kilosort_if_exists-norm=None-combine=True-RAW.pkl" # Novel prims.
DFallpa = pd.read_pickle(path)


In [ ]:
dflab = DFallpa["labels"].values[0]


In [ ]:
sorted(dflab["seqc_0_shape"].unique().tolist())

In [ ]:
import random
s = "asdfdasfdsafsadfsdafsaddf1- 123 130i23 ."
r = random.Random(s); 
"".join([str(r.randrange(10)) for _ in range(10)])

# Load a dataset

To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper
animal = "Diego"
date = 230615
version = "trial"
combine_areas = True
load_spike_counts_version = True
DFallpa = load_handsaved_wrapper(animal, date, version, combine_areas, load_spike_counts_version=True)

AssertionError: to not run into error of loading old pa

##### [Optionally, merge across events]

In [ ]:
DFallpa

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_group_and_split

DFallpa = dfpa_group_and_split(DFallpa, ["event"], concat_dim="trials")

In [ ]:
DFallpa

In [ ]:
pa = DFallpa["pa"].values[0]
pa.Xlabels["trials"]["event"].value_counts()
pa.plotNeurHeat(0)

##### Preprocess fr normalization?

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
dfallpa_preprocess_fr_normalization(DFallpa, "across_time_bins")

# [LFADS] Prep dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper
animal = "Diego"
date = 230615
version = "trial"
question = "SP_BASE_trial"
combine_areas = True
load_spike_counts_version = True
DFallpa = load_handsaved_wrapper(animal, date, version, combine_areas, question=question, load_spike_counts_version=True)

Loading DFallpa from:  /lemur2/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-230615-trial-kilosort_if_exists-norm=None-combine=True-t1=-1.0-t2=1.8-quest=SP_BASE_trial-spkcnts_binsz=0.01.pkl


In [9]:
import h5py


In [10]:
# # Extract data in this format

# # Basic Walkthrough
# ## DataModule Configuration
# The first step in applying `lfads-torch` to your dataset is to prepare your preprocessed data files. Save your data as `n_samples x n_timesteps x n_channels` arrays in the HDF5 format using the following keys:
# - `train_encod_data`: Data to be used as input when training the model.
# - `train_recon_data`: Data to be used as a reconstruction target when training the model.
# - `valid_encod_data`: Data to be used as input when validating the model.
# - `valid_recon_data`: Data to be used as a reconstruction target when validating the model.

# Note that for both training and validation data, `encod_data` may be the same as `recon_data`, but they can be different to allow prediction of held out neurons or time steps.

# Create a new configuration file for your dataset (e.g. `configs/datamodule/my_datamodule.yaml`). For single-session runs, set `datafile_pattern` to the path to your data file. For multi-session runs, set `datafile_pattern` to a `glob`-style pattern that matches all of your data files.
# ```
# _target_: lfads_torch.datamodules.BasicDataModule
# datafile_pattern: <PATH-TO-HDF5-FILE>
# batch_size: <YOUR-BATCH-SIZE>
# ```

# We provide preprocessed example data files from the Neural Latents Benchmark in `datasets`. With [`nlb_tools`](https://github.com/neurallatents/nlb_tools) installed in your environment, you can additionally use the `NLBEvaluation` extension to monitor NLB metrics while training `lfads-torch` models.


In [15]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,trial,03_samp,M1,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(46, 425, 280)"
1,trial,03_samp,PMv,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(70, 425, 280)"
2,trial,03_samp,PMd,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(37, 425, 280)"
3,trial,03_samp,dlPFC,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 425, 280)"
4,trial,03_samp,vlPFC,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(54, 425, 280)"
5,trial,03_samp,FP,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(35, 425, 280)"
6,trial,03_samp,SMA,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(53, 425, 280)"
7,trial,03_samp,preSMA,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(68, 425, 280)"
8,trial,05_first_raise,M1,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(46, 425, 280)"
9,trial,05_first_raise,PMv,"(-1.0, 1.8)",<neuralmonkey.classes.population.PopAnal objec...,"(70, 425, 280)"


In [ ]:
animal = "Diego"
date = 230615
version = "trial"
question = "SP_BASE_trial"
combine_areas = True
load_spike_counts_version = True


'Diego'

In [ ]:
SAVEDIR = f"/lemur2/lucas/neural_preprocess/LFADS/{animal}-{date}-{version}-{question}-combine={combine_areas}"
os.makedirs(SAVEDIR, exist_ok=True)
import numpy as np

for i, row in DFallpa.iterrows():
    
    bregion = row["bregion"]
    event = row["event"]
    PA = row["pa"]
    X = np.transpose(PA.X, axes=(1, 2, 0)) # n_samples(ntrials) x n_timesteps x n_channels
    import h5py
    # 'train_encod_data`: Data to be used as input when training the model.
    # `train_recon_data`: Data to be used as a reconstruction target when training the model.
    # `valid_encod_data`: Data to be used as input when validating the model.
    # `valid_recon_data`: Data to be used as a reconstruction target when validating the model.

    savepath = f"{SAVEDIR}/{bregion}-{event}.h5"
    print(savepath)
    with h5py.File(savepath, 'w') as hdf:
        hdf.create_dataset('train_encod_data', data=X)
        hdf.create_dataset('train_recon_data', data=X)
        hdf.create_dataset('valid_encod_data', data=X)
        hdf.create_dataset('valid_recon_data', data=X)
        # hdf.create_dataset('train_labels', data=train_label_array)
        # hdf.create_dataset('holdout_labels', data=holdout_label_array)
        # hdf.create_dataset('test_data', data=reshaped_test)
        # hdf.create_dataset('test_labels', data=test_label_array)

    with h5py.File(savepath, 'r') as hdf:
        data1 = hdf['train_encod_data']
        data2 = hdf['valid_encod_data']
        # data5 = hdf['test_data']
        print(data1.shape)
        print(data2.shape)
        # print(data5.shape)
    path = "/home/lucas/code/lfads-torch-clean/datasets/mc_maze_small-05ms-val.h5"
    with h5py.File(path, "r") as hdf:
        print(hdf.keys())
        print(hdf["train_encod_data"])

In [17]:
# NOTES:

In [ ]:
# The spiking data were binned at 1 ms and smoothed by convolution with a Gaussian kernel (30 ms s.d.).

# 700ms window.

# Datasets of varying sizes were created for LFADS by randomly selecting trials with 20%, 10% and 5% of the original dataset using seven fixed seeds, and then splitting each of these into 80/20 training and validation sets for LFADS (22 total, including the full dataset).

# Remove correlated neurons
# We started with sorted units obtained from M1 and binned their spike times at 1 ms. To avoid artifacts in which the same spikes appeared on multiple channels, we computed cross-correlations between all pairs of neurons over the first 10 s and removed individual correlated neurons (n = 34) by highest firing rate until there were no pairs with correlation values above 0.0625, resulting in 181 uncorrelated neurons. We remove these neurons because correlated spike artifacts can cause overfitting issues, despite the protection afforded by CD.

In [ ]:
# Tutorial: https://snel-repo.github.io/autolfads/
